In [1]:
from pyspark.sql import SparkSession
def get_spark():
    spark = SparkSession.builder.appName('SparkDelta') \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.jars.packages",
                "org.apache.hadoop:hadoop-aws:3.2.2,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.180") \
        .getOrCreate()
# This is mandate config on spark session to use AWS S3
    spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
    spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")
# spark.sparkContext.setLogLevel("DEBUG")
    return spark
spark = get_spark()
print("Spark session created")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1666882784054_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark session created

In [13]:
category_path = "s3a://youtube-dataset-kshitij/output/category/part*"
videos_path = "s3a://youtube-dataset-kshitij/output/videos/part*"
category_df = spark.read.parquet(category_path)
videos_df = spark.read.parquet(videos_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
print("Total number of records in category_df is - ",category_df.count())
print("Total number of records in videos_df is - ",videos_df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of records in category_df is -  32
Total number of records in videos_df is -  178363

In [15]:
category_df.show()
videos_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+
| id|               title|
+---+--------------------+
| 31|     Anime/Animation|
|  2|    Autos & Vehicles|
| 44|            Trailers|
| 39|              Horror|
| 19|     Travel & Events|
| 30|              Movies|
| 36|               Drama|
| 18|        Short Movies|
| 27|           Education|
| 21|       Videoblogging|
| 33|            Classics|
| 24|       Entertainment|
| 28|Science & Technology|
| 32|    Action/Adventure|
| 22|      People & Blogs|
| 38|             Foreign|
| 10|               Music|
| 42|              Shorts|
| 17|              Sports|
|  1|    Film & Animation|
+---+--------------------+
only showing top 20 rows

+-----------+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+
|category_id|               title|       channel_title|publish_date|       publish_time|trending_date|  views| likes|dislikes|comment_count|
+-----------+--------------------+----------

In [16]:
#Renaming column title in category_df as category_title since 'title' column also exists in videos_df 
category_df = category_df.withColumnRenamed("title","category_title")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
import pyspark.sql.functions as f
#Applying broadcast join on category_df as it has only 32 records
joined_df = videos_df.join(f.broadcast(category_df),category_df.id==videos_df.category_id,"inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
joined_df = joined_df.drop('id','category_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
joined_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|               title|       channel_title|publish_date|       publish_time|trending_date|  views| likes|dislikes|comment_count|      category_title|
+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|WE WANT TO TALK A...|        CaseyNeistat|  2017-11-13|2017-11-13 17:13:01|   2017-11-14| 748374| 57527|    2966|        15954|      People & Blogs|
|The Trump Preside...|     LastWeekTonight|  2017-11-13|2017-11-13 07:30:00|   2017-11-14|2418783| 97185|    6146|        12703|       Entertainment|
|Racist Superman |...|        Rudy Mancuso|  2017-11-12|2017-11-12 19:05:24|   2017-11-14|3191434|146033|    5339|         8181|              Comedy|
|Nickelback Lyrics...|Good Mythical Mor...|  2017-11-13|2017-11-13 11:00:04|   2017-11-14| 343168| 1

In [22]:
#1. Determine the most liked video
most_liked_video_df = joined_df.where(joined_df.agg(f.max("likes").alias("max_likes")).collect()[0][0]==joined_df.likes)
most_liked_video_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+-------------+------------+-------------------+-------------+---------+-------+--------+-------------+--------------+
|                    title|channel_title|publish_date|       publish_time|trending_date|    views|  likes|dislikes|comment_count|category_title|
+-------------------------+-------------+------------+-------------------+-------------+---------+-------+--------+-------------+--------------+
|BTS (방탄소년단) 'FAKE...|      ibighit|  2018-05-18|2018-05-18 09:00:02|   2018-06-01|123010920|5613827|  206892|      1228655|         Music|
+-------------------------+-------------+------------+-------------------+-------------+---------+-------+--------+-------------+--------------+

In [23]:
#Validating the results if max liked video is correct in above result by ordering the joined_df in desc order of likes
validate_max_liked_video  = joined_df.orderBy(joined_df.likes.desc())
validate_max_liked_video.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+-------------+------------+-------------------+-------------+---------+-------+--------+-------------+--------------+
|                    title|channel_title|publish_date|       publish_time|trending_date|    views|  likes|dislikes|comment_count|category_title|
+-------------------------+-------------+------------+-------------------+-------------+---------+-------+--------+-------------+--------------+
|BTS (방탄소년단) 'FAKE...|      ibighit|  2018-05-18|2018-05-18 09:00:02|   2018-06-01|123010920|5613827|  206892|      1228655|         Music|
|BTS (방탄소년단) 'FAKE...|      ibighit|  2018-05-18|2018-05-18 09:00:02|   2018-05-31|121219886|5595203|  205565|      1225326|         Music|
|BTS (방탄소년단) 'FAKE...|      ibighit|  2018-05-18|2018-05-18 09:00:02|   2018-05-30|115664850|5530568|  200995|      1213172|         Music|
|BTS (방탄소년단) 'FAKE...|      ibighit|  2018-05-18|2018-05-18 09:00:02|   2018-05-29|111882133|5486349|  197638|      1204867|         Music|
|BTS 

In [25]:
#2. Determine the most commented video
most_commented_video_df = joined_df.where(joined_df.agg(f.max("comment_count").alias("max_comments")).collect()[0][0]==joined_df.comment_count)
most_commented_video_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+------------+-------------------+-------------+--------+-------+--------+-------------+--------------+
|    title|   channel_title|publish_date|       publish_time|trending_date|   views|  likes|dislikes|comment_count|category_title|
+---------+----------------+------------+-------------------+-------------+--------+-------+--------+-------------+--------------+
|So Sorry.|Logan Paul Vlogs|  2018-01-02|2018-01-02 16:42:21|   2018-01-09|37539570|1402578| 1674420|      1361580| Entertainment|
+---------+----------------+------------+-------------------+-------------+--------+-------+--------+-------------+--------------+

In [26]:
joined_df = joined_df.withColumnRenamed("category_title","category")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
joined_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|               title|       channel_title|publish_date|       publish_time|trending_date|  views| likes|dislikes|comment_count|            category|
+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|WE WANT TO TALK A...|        CaseyNeistat|  2017-11-13|2017-11-13 17:13:01|   2017-11-14| 748374| 57527|    2966|        15954|      People & Blogs|
|The Trump Preside...|     LastWeekTonight|  2017-11-13|2017-11-13 07:30:00|   2017-11-14|2418783| 97185|    6146|        12703|       Entertainment|
|Racist Superman |...|        Rudy Mancuso|  2017-11-12|2017-11-12 19:05:24|   2017-11-14|3191434|146033|    5339|         8181|              Comedy|
|Nickelback Lyrics...|Good Mythical Mor...|  2017-11-13|2017-11-13 11:00:04|   2017-11-14| 343168| 1

In [31]:
#3. Category wise most liked video along with channel name
from pyspark.sql.window import Window
partition=Window.partitionBy("category").orderBy(joined_df.likes.desc())
category_most_liked_video=joined_df.withColumn("dense_r",f.dense_rank().over(partition)).filter(f.col('dense_r')<=1). \
    select("category","channel_title","likes")
category_most_liked_video.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------+
|            category|       channel_title|  likes|
+--------------------+--------------------+-------+
|    Autos & Vehicles|            CHRIS-RS|  97030|
|    Film & Animation|        Selena Gomez| 922355|
|       Howto & Style|               Vogue| 605954|
|Science & Technology|         AsapSCIENCE| 564804|
|              Sports|        Desimpedidos|1121217|
|       Entertainment|   YouTube Spotlight|3093544|
|      People & Blogs|        David Dobrik|2032463|
|           Education|           PewDiePie| 352049|
|              Movies|         Rhythm Boyz|  77789|
|     News & Politics|           PewDiePie| 552150|
|Nonprofits & Acti...|    Logan Paul Vlogs|1988746|
|     Travel & Events|         LE GRAND JD| 150626|
|              Comedy|           Lele Pons| 865679|
|      Pets & Animals|           PewDiePie| 193092|
|            Trailers|La Malédiction Wi...|    138|
|              Gaming|       jacksepticeye| 444203|
|           

In [40]:
#4. Display channel wise total (aggregated) likes in desc order
channel_total_likes_desc = joined_df.groupby("channel_title").agg(f.sum("likes").alias("total_likes"))
channel_total_likes_desc = channel_total_likes_desc.orderBy(channel_total_likes_desc.total_likes.desc())
channel_total_likes_desc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|       channel_title|total_likes|
+--------------------+-----------+
|             ibighit|  283786333|
|Marvel Entertainment|  138062378|
|        Dude Perfect|  125369809|
| ChildishGambinoVEVO|  117527286|
|           PewDiePie|   87971985|
|     TaylorSwiftVEVO|   86585685|
|          Ed Sheeran|   80762668|
|    ArianaGrandeVevo|   67300244|
|        Amit Bhadana|   64203102|
|    Logan Paul Vlogs|   63350859|
|Sony Pictures Ent...|   62059534|
|   YouTube Spotlight|   58686848|
|        FoxStarHindi|   58201569|
|            T-Series|   55807729|
|    jypentertainment|   54729095|
|            ZaynVEVO|   52549017|
|            nigahiga|   49055598|
|          EminemVEVO|   47398810|
|    20th Century Fox|   46762012|
|             Cardi B|   43368065|
+--------------------+-----------+
only showing top 20 rows

In [42]:
#5. Display CATEGORY wise total (aggregated) likes in desc order
category_total_likes_desc = joined_df.groupby("category").agg(f.sum("likes").alias("total_likes"))
category_total_likes_desc = category_total_likes_desc.orderBy(category_total_likes_desc.total_likes.desc())
category_total_likes_desc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|            category|total_likes|
+--------------------+-----------+
|               Music| 2512928884|
|       Entertainment| 1432152121|
|              Comedy|  717313550|
|      People & Blogs|  344126997|
|    Film & Animation|  337672454|
|       Howto & Style|  237472022|
|              Sports|  235005176|
|Science & Technology|  165256446|
|              Gaming|  149975268|
|           Education|   86004544|
|     News & Politics|   75702100|
|Nonprofits & Acti...|   31263755|
|      Pets & Animals|   29899421|
|    Autos & Vehicles|   11124108|
|     Travel & Events|    9475873|
|               Shows|    2750630|
|              Movies|     939851|
|            Trailers|        192|
+--------------------+-----------+

In [43]:
#6. Top 5 popular categories liked by people
category_total_likes_desc = category_total_likes_desc.limit(5)
category_total_likes_desc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|        category|total_likes|
+----------------+-----------+
|           Music| 2512928884|
|   Entertainment| 1432152121|
|          Comedy|  717313550|
|  People & Blogs|  344126997|
|Film & Animation|  337672454|
+----------------+-----------+

In [44]:
#7. Top 5 most viewed categories of videos
category_total_views_desc = joined_df.groupby("category").agg(f.sum("views").alias("total_views"))
category_total_views_top_5 = category_total_views_desc.orderBy(category_total_views_desc.total_views.desc()).limit(5)
category_total_views_top_5.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|        category|total_views|
+----------------+-----------+
|           Music|68490034391|
|   Entertainment|54175494907|
|Film & Animation|14988884087|
|          Comedy|13609821712|
|  People & Blogs|10790367427|
+----------------+-----------+

In [45]:
#8. Second most commented video of each category
partition=Window.partitionBy("category").orderBy(joined_df.comment_count.desc())
second_most_commented_video_in_category = joined_df.withColumn("dense_r",f.dense_rank().over(partition)). \
    filter(f.col('dense_r')==2). \
    select("category","channel_title","comment_count")
second_most_commented_video_in_category.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+-------------+
|            category|    channel_title|comment_count|
+--------------------+-----------------+-------------+
|    Autos & Vehicles|         ChrisFix|        25643|
|    Film & Animation|Jaiden Animations|       105594|
|       Howto & Style|      jeffreestar|       262658|
|Science & Technology| Technical Guruji|       377912|
|              Sports|     Desimpedidos|        79844|
|       Entertainment| Logan Paul Vlogs|      1321281|
|      People & Blogs|   Safiya Nygaard|       378185|
|           Education|  #Mind Warehouse|        85367|
|              Movies|      Rhythm Boyz|         4279|
|     News & Politics|        PewDiePie|        61795|
|Nonprofits & Acti...| Logan Paul Vlogs|       647524|
|     Travel & Events|      LE GRAND JD|        14607|
|              Comedy|        PewDiePie|        83116|
|      Pets & Animals| Brave Wilderness|        44132|
|              Gaming|     Call of Duty|       144537|
|         

In [48]:
joined_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|               title|       channel_title|publish_date|       publish_time|trending_date|  views| likes|dislikes|comment_count|            category|
+--------------------+--------------------+------------+-------------------+-------------+-------+------+--------+-------------+--------------------+
|WE WANT TO TALK A...|        CaseyNeistat|  2017-11-13|2017-11-13 17:13:01|   2017-11-14| 748374| 57527|    2966|        15954|      People & Blogs|
|The Trump Preside...|     LastWeekTonight|  2017-11-13|2017-11-13 07:30:00|   2017-11-14|2418783| 97185|    6146|        12703|       Entertainment|
|Racist Superman |...|        Rudy Mancuso|  2017-11-12|2017-11-12 19:05:24|   2017-11-14|3191434|146033|    5339|         8181|              Comedy|
|Nickelback Lyrics...|Good Mythical Mor...|  2017-11-13|2017-11-13 11:00:04|   2017-11-14| 343168| 1

In [63]:
#9. Which video gained most likes which trended same day of being published
days_trending_df = joined_df.withColumn("trending_days",f.datediff("trending_date","publish_date"))
same_day_trended_videos = days_trending_df.filter(days_trending_df.trending_days==0)
same_day_trended_most_liked_video = same_day_trended_videos.orderBy(days_trending_df.likes.desc()).limit(1)
same_day_trended_most_liked_video.select("title","category","likes","channel_title").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+--------+------+-------------+
|title                               |category|likes |channel_title|
+------------------------------------+--------+------+-------------+
|Bad Bunny - Amorfoda | Video Oficial|Music   |544514|Bad Bunny    |
+------------------------------------+--------+------+-------------+

In [66]:
#10. Top 5 categories whose videos went trending same day the most number of times
category_same_day_trending = same_day_trended_videos.groupby("category"). \
    agg(f.count("trending_days").alias("total_trended"))
top_5_category_same_day_trending = category_same_day_trending.orderBy(f.desc("total_trended"))
top_5_category_same_day_trending.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+
|       category|total_trended|
+---------------+-------------+
|  Entertainment|          845|
|         Sports|          623|
|News & Politics|          490|
| People & Blogs|          338|
|          Music|          276|
+---------------+-------------+
only showing top 5 rows